In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

file_path = 'Copy of Claims Minified Data.xlsx' 
df = pd.read_excel(file_path, sheet_name='Sheet4', engine='openpyxl')

print(f"Original data loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# = 2. Clean column names 
df.columns = df.columns.str.strip().str.replace('##', '')

#  3. Handle missing values 
df = df.replace(['N/A', '', 'NA'], np.nan)

# =4. Parse all date columns =
date_cols = ['REGISTRATION_DATE', 'KC_REPUDIATE_DT', 'NOTIFICATE_DATE', 
             'DATE_LOSS', 'NOTIFICATION_DATE', 'COVER_START_DATE', 'COVER_END_DATE']

common_formats = ['%d-%b-%Y', '%d-%B-%Y', '%d-%m-%Y', '%Y-%m-%d', '%d/%m/%Y']

for col in date_cols:
    if col in df.columns:
        parsed = pd.Series([pd.NaT] * len(df))  
        success = False
        for fmt in common_formats:
            try:
                # Adjust dayfirst based on format
                dayfirst = '%' not in fmt.split('-')[0]  
                temp = pd.to_datetime(df[col], format=fmt, errors='raise', dayfirst=dayfirst)
                parsed = temp
                success = True
                break
            except:
                continue
        if not success:
            
            parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
        df[col] = parsed
        
# = 5. Clean numeric columns 
numeric_cols = ['DRIVER_AGE', 'KC_OUTSTANDING_RESERVE', 'TIME_TAKEN_REGISTRATION',
                'KC_STATUS', 'KC_SUM_INSURED', 'MANUFACTURING_YEAR', 'POLICY_HOLDER_AGE']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

# = 6. Fix age anomalies (negative or zero) 
age_cols = ['DRIVER_AGE', 'POLICY_HOLDER_AGE']
for col in age_cols:
    if col in df.columns:
        df[col] = df[col].where(df[col] > 0, np.nan)
        median_age = df[col].median()
        df[col] = df[col].fillna(median_age)

# Impute MANUFACTURING_YEAR with median if needed
if 'MANUFACTURING_YEAR' in df.columns:
    df['MANUFACTURING_YEAR'] = df['MANUFACTURING_YEAR'].fillna(df['MANUFACTURING_YEAR'].median())

# = 7. Clean categorical columns =
cat_cols = ['KC_STATUS_TEXT', 'POLICE_REPORTED', 'PERILNAME', 'IC_NAME', 'GP_NAME',
            'GCT_NAME', 'INT_TYP', 'BR_NAME', 'MAKE_NAME', 'MODEL_NAME',
            'POLICY_HOLDER_GENDER', 'POLICE_STATION_NAME']

for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.title()
        df[col] = df[col].replace(['Nan', 'Na'], np.nan)

# Special handling
df['POLICE_STATION_NAME'] = df['POLICE_STATION_NAME'].fillna('Unknown')
df['POLICE_REPORTED'] = df['POLICE_REPORTED'].replace({np.nan: 'No', 'Yes': 'Yes', 'No': 'No'})

# ==================== 8. Repudiation remarks ===
repud_cols = ['KC_FIRST_REPUDIATE_REM', 'KC_FIRST_REPUDIATE_REM_DATA',
              'KC_SECOND_REPUDIATE_REM', 'KC_FIFTH_REPUDIATE_REM']
for col in repud_cols:
    if col in df.columns:
        df[col] = df[col].fillna('No Repudiation')

# ==================== 9. Drop duplicates ====================
initial_rows = df.shape[0]
df = df.drop_duplicates(subset=['CLAIMNUMBER'], keep='first')
print(f"Duplicates removed: {initial_rows - df.shape[0]}")

# ==================== 10. Final touches ====================
# Ensure ID columns are strings
id_cols = ['POLICY_NUMBER', 'CLAIMNUMBER', 'CLIENT_MOBILE_NUMBER', 'INTERMEDIARY_PHONE_NO']
for col in id_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)

# Drop completely empty columns if any
df = df.dropna(axis=1, how='all')

# ==================== 11. Add analysis-ready columns ====================
df['Date_Reported'] = df['NOTIFICATION_DATE']  # Best proxy for when claim was reported
df['Quarter'] = df['Date_Reported'].dt.to_period('Q').astype(str)
df['Year'] = df['Date_Reported'].dt.year

# ==================== 12. Save cleaned file ====================
output_file = 'fully_cleaned_claims_data.csv'
df.to_csv(output_file, index=False)
print(f"\nCleaning complete! Cleaned data saved to: {output_file}")
print(f"Final shape: {df.shape[0]} rows, {df.shape[1]} columns")

# ==================== 13. Quick overview ====================
print("\nFirst 5 rows:")
print(df.head())

print("\nMissing values summary:")
print(df.isnull().sum()[df.isnull().sum() > 0])

print("\nDate columns info:")
print(df[['REGISTRATION_DATE', 'NOTIFICATION_DATE', 'DATE_LOSS', 'Date_Reported', 'Quarter']].head(10))

Original data loaded: 1039 rows, 42 columns
Duplicates removed: 0

Cleaning complete! Cleaned data saved to: fully_cleaned_claims_data.csv
Final shape: 1039 rows, 42 columns

First 5 rows:
  REGISTRATION_DATE KC_REPUDIATE_DT  DRIVER_AGE  KC_OUTSTANDING_RESERVE  \
0        2025-01-06             NaT        31.0                       0   
1        2024-11-29             NaT        31.0                     700   
2        2025-03-21             NaT        31.0                  120700   
3        2024-12-13             NaT        31.0                       0   
4        2025-02-24             NaT        31.0                   80200   

   TIME_TAKEN_REGISTRATION  KC_STATUS      KC_STATUS_TEXT POLICY_NUMBER  \
0                        1         74  Authorized Payment   G/HQS/00001   
1                        1          2           Inprocess   G/HQS/00002   
2                        1          5           Inprocess   G/EMB/00003   
3                        1          2           Inprocess   

C:\Users\HP\AppData\Local\Temp\ipykernel_13252\1238469881.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13252\1238469881.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13252\1238469881.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors='coerce', dayfirst=True)


KC_REPUDIATE_DT         1002
PERILNAME                  1
GCT_NAME                   3
MAKE_NAME                106
MODEL_NAME               106
POLICY_HOLDER_GENDER     209
dtype: int64

Date columns info:
  REGISTRATION_DATE NOTIFICATION_DATE  DATE_LOSS Date_Reported Quarter
0        2025-01-06        2025-01-06 2024-12-24    2025-01-06  2025Q1
1        2024-11-29        2024-11-29 2024-08-22    2024-11-29  2024Q4
2        2025-03-21        2025-03-21 2024-02-28    2025-03-21  2025Q1
3        2024-12-13        2024-12-13 2023-05-02    2024-12-13  2024Q4
4        2025-02-24        2025-02-24 2023-06-26    2025-02-24  2025Q1
5        2025-03-19        2025-03-19 2024-02-17    2025-03-19  2025Q1
6        2025-02-05        2025-02-05 2022-02-12    2025-02-05  2025Q1
7        2025-02-21        2025-02-21 2022-08-29    2025-02-21  2025Q1
8        2025-01-31        2025-01-31 2024-09-18    2025-01-31  2025Q1
9        2025-03-12        2025-03-12 2023-09-23    2025-03-12  2025Q1


In [4]:
df.to_excel('fully_cleaned_claims_data.xlsx', index=False)